In [19]:
%matplotlib inline

import seaborn as sns
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.feature_selection import VarianceThreshold, f_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from numpy import corrcoef, sum, log, arange
from pylab import pcolor, show, colorbar, xticks, yticks
import seaborn as sns
import pandas as pd
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans,MiniBatchKMeans
from scipy.stats import pearsonr
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.externals import joblib
from sklearn.decomposition import PCA, RandomizedPCA, KernelPCA
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.learning_curve import learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.externals import joblib
import pickle
import os

import sys
sys.path.insert(1, "/home/aegle/projects/myosa/src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score

#### 1) Compute & save TRAIN, TEST datasets

In [22]:
for suffix in suffixes:

    #Load clean data
    fileNames = []
    fileNames.append(os.path.join('resources','data_filtered_cl0_short_no_monit_14012016_final.pkl'))
    fileNames.append(os.path.join('resources','data_filtered_cl0_short_1m_monit_14012016_final.pkl'))
    fileNames.append(os.path.join('resources','data_filtered_cl0_short_3m_1m_monit_14012016_final.pkl'))

    df_filtered = []
    for f in fileNames:
        print f
        df_filtered.append(pd.read_pickle(f))

    saveFileNames = []
    saveFileNames.append(os.path.join('resources','data_partition_cl0_short_no_monit_14012016.pkl'))
    saveFileNames.append(os.path.join('resources','data_partition_cl0_short_1m_monit_14012016.pkl'))
    saveFileNames.append(os.path.join('resources','data_partition_cl0_short_3m_1m_monit_14012016.pkl'))

    for k,lf in enumerate(saveFileNames):

        print "\nPROCESSING-{} :".format(k,lf)

        #Split features - label
        df = df_filtered[k]#.values[:,1:-1]
        all_columns = df.columns[1:-1] 
        
        #Split features - label
        X = df_filtered[k].values[:,1:-1]
        y = df_filtered[k].values[:,-1].astype(int)          
        
        y[y < 4.0] = 0
        y[y >= 4.0] = 1
        

resources/data_filtered_cl0_short_no_monit_14012016_final.pkl
resources/data_filtered_cl0_short_1m_monit_14012016_final.pkl
resources/data_filtered_cl0_short_3m_1m_monit_14012016_final.pkl

PROCESSING-0 :

PROCESSING-1 :

PROCESSING-2 :


In [24]:
for k,lf in enumerate(saveFileNames):

    print "\nPROCESSING-{} :".format(k,lf)

    #Split features - label
    X = df_filtered[k].values[:,1:-1]
    y = df_filtered[k].values[:,-1].astype(int)          

    y[y <= 4.0] = 0
    y[y > 4.0] = 1
    
    feat_type = dict([(c,df_filtered[k][c].dtype.name) for c in df_filtered[k].columns])
    print feat_type

    #print df_filtered[k].info()
    print "X", X.shape, "Y", y.shape
    print "Num 0s: ", np.sum(y==0), "Num 1s: ", np.sum(y==1)

    #Compute partition (train, test)
    sss = StratifiedShuffleSplit(y, 1, test_size=0.3, random_state=32) #random_state=42
    for train_index, test_index in sss:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_tr_codes = df_filtered[k]['CODIGO'].ix[train_index].values
        X_ts_codes = df_filtered[k]['CODIGO'].ix[test_index].values
        print X_tr_codes
        print X_ts_codes

    f = file(saveFileNames[k],"wb")
    np.save(f, X_train)
    np.save(f, y_train)
    np.save(f, X_test)
    np.save(f, y_test)
    np.save(f, df_filtered[k].columns[1:])
    np.save(f,X_tr_codes)
    np.save(f,X_ts_codes)
    np.save(f,feat_type)
    f.close()

    f = file(saveFileNames[k],"rb")
    aa = np.load(f)
    bb = np.load(f)
    cc = np.load(f)
    dd = np.load(f)
    ee = np.load(f)
    ff = np.load(f)
    gg = np.load(f)
    hh = np.load(f)
    f.close()

    print aa.shape, bb.shape, cc.shape, dd.shape, ee.shape, ff.shape, gg.shape
    print "\nTRAIN:  Num 0s: ", np.sum(bb==0), "Num 1s: ", np.sum(bb==1)
    print "TEST: Num 0s: ", np.sum(dd==0), "Num 1s: ", np.sum(dd==1)


PROCESSING-0 :
{'ANXIETY': 'object', 'ALCOHOL': 'object', 'SMOKE_EX_YEAR': 'float64', 'CARDIOPATHY': 'object', 'SAT_O2': 'int64', 'CRISIS_ASP': 'int64', 'AGE': 'int64', 'CEFALEA': 'int64', 'DISLIPEMIA': 'object', 'SNORES': 'int64', 'TAD_MEAN': 'float64', 'DEPRESSION': 'object', 'ALCOHOL_GR_DAY': 'float64', 'CAFES_DAY': 'float64', 'ANTIAGREG': 'object', 'APNEAS_PRES': 'float64', 'CIRC_NECK': 'int64', 'GENDER': 'object', 'EPWORTH': 'float64', 'BMI': 'float64', 'REP_SLEEP': 'int64', 'ACT_MOTR_NOCT': 'object', 'SMOKE_PACK_YEAR': 'int64', 'IECAS': 'object', 'DAY_SOMN': 'int64', 'HIPOLIPEM': 'object', 'CODIGO': 'object', 'DIURETICS': 'object', 'IAH': 'float64', 'FEEL_REAL_DRE': 'object', 'IND_DESAT': 'float64', 'INSOMNIA': 'object', 'VISUAL': 'int64', 'RESP_DIS': 'object', 'AWAKE_NIGHT': 'int64', 'IRR_APAT_DEPRE': 'int64', 'ACTIVE': 'object', 'OBESITY': 'object', 'LABEL': 'float64', 'SLEEP_TIME': 'float64', 'CONC_DIS': 'int64', 'DIABETES': 'object', 'RETIRED': 'object', 'TIME_SAT<90%': 'flo

### Compare TR vs TS

In [ ]:
catCols = []
reducedCols = columns[:-1]

for i in range(len(columns)-1):

    if feat_types[columns[i]] == 'object' \
        or columns[i] in ("RONCA","SUE_REP","CRISIS_ASF","NICTURIA","APNEAS_PRES","CEFALEA",
                "SOMN_DIURN","DESPERTAR_NOCT","TRAST_MEM","TRAST_CONC","IRR_APAT_DEPR"):
        catCols.append(1)
    else:
        catCols.append(0)
catCols = np.array(catCols)

print "Cat cols:", np.sum(catCols==1)
print "Num cols:", np.sum(catCols==0)
print len(reducedCols)

In [ ]:
#Transform discrete
imputer = TypeFeatImputer(catCols, reducedCols)
imputer.fit(df_train.iloc[:,:-1].values.astype(float),df_train.iloc[:,-1].values.astype(int))

df_tr = imputer.transform(df_train.iloc[:,:-1].values.astype(float))
df_ts = imputer.transform(df_test.iloc[:,:-1].values.astype(float))

In [ ]:
scaler = preprocessing.StandardScaler().fit(df_tr)
df_tr_scaled = scaler.transform(df_tr)
df_tr_scaled = pd.DataFrame(
    data = np.hstack((df_tr_scaled, df_train.ix[:,-1].values.reshape((df_train.shape[0],1)))), 
    columns = df_train.columns)

df_ts_scaled = scaler.transform(df_ts)
df_ts_scaled = pd.DataFrame(
    data = np.hstack((df_ts_scaled, df_test.ix[:,-1].values.reshape((df_test.shape[0],1)))), 
    columns = df_test.columns)

print "DF_TRAIN scaled", df_tr_scaled.shape, type(df_tr_scaled)
print "DF_TEST scaled", df_ts_scaled.shape, type(df_ts_scaled)

In [ ]:
tr_means_0 = df_tr_scaled[df_tr_scaled['LABEL']==0].iloc[:,:-1].astype(float).iloc[:,catCols == 0].describe()\
            .loc[['mean']]
ts_means_0 = df_ts_scaled[df_ts_scaled['LABEL']==0].iloc[:,:-1].astype(float).iloc[:,catCols == 0].describe()\
            .loc[['mean']]

tr_means_1 = df_tr_scaled[df_tr_scaled['LABEL']==1].iloc[:,:-1].astype(float).iloc[:,catCols == 0].describe()\
            .loc[['mean']]
ts_means_1 = df_ts_scaled[df_ts_scaled['LABEL']==1].iloc[:,:-1].astype(float).iloc[:,catCols == 0].describe()\
            .loc[['mean']]

print "Total columns:", tr_means_0.shape

# Loop by column and store if TS_0 has fewer distance with TR_0 rather than with TR_1
colsSameTrend = []
for i in range(tr_means_0.shape[1] - 1):
        
    colsSameTrend.append([
            tr_means_0.columns[i],
            tr_means_0.ix[0,i], 
            tr_means_1.ix[0,i], 
            ts_means_0.ix[0,i],
            ts_means_1.ix[0,i],
            (tr_means_0.ix[0,i] - ts_means_0.ix[0,i]), 
            (tr_means_1.ix[0,i] - ts_means_1.ix[0,i]),
            abs((tr_means_0.ix[0,i] - ts_means_0.ix[0,i]) - (tr_means_1.ix[0,i] - ts_means_1.ix[0,i]))
        ])   

all_trends = np.array(colsSameTrend)

display(pd.DataFrame(all_trends, 
                     columns=["name","mean_tr_0","mean_tr_1","mean_ts_0","mean_ts_1",
                              "diff_tr_ts_0","diff_tr_ts_1","diff_tr_tr_all"]))

print "Number of columns with fewer distances with TR_0 than TR_1:", all_trends.shape, "from", tr_means_0.shape[1]
print "Number of columns with fewer distances with TR_0 than TR_1 > 0.5:", trends.shape, "from", tr_means_0.shape[1]


#trends = all_trends[all_trends[:,7].astype(float) > 0.5,:] 
#dfTrends = pd.DataFrame(data = trends[:,1:].T, columns=trends[:,0])
#dfTrends.ix[(0,1,2,5),:]